In [21]:
import numpy as np
import pandas as pd
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf 
from statsmodels.stats.outliers_influence import variance_inflation_factor 
from statsmodels.tools.tools import add_constant
from sklearn import preprocessing
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf 
from statsmodels.stats.outliers_influence import variance_inflation_factor 
from statsmodels.tools.tools import add_constant
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import colors
import seaborn as sn
from scipy.stats import pearsonr

In [22]:
POP = pd.read_csv('https://raw.githubusercontent.com/XianlaiYin/Data/main/CASA0007/data_clean/POP.csv')
GDR = pd.read_csv('https://raw.githubusercontent.com/XianlaiYin/Data/main/CASA0007/data_clean/GDR.csv')
URB = pd.read_csv('https://raw.githubusercontent.com/XianlaiYin/Data/main/CASA0007/data_clean/URB.csv')
EDU = pd.read_csv('https://raw.githubusercontent.com/XianlaiYin/Data/main/CASA0007/data_clean/EDU.csv')
FAM = pd.read_csv('https://raw.githubusercontent.com/XianlaiYin/Data/main/CASA0007/data_clean/FAM.csv')
GEN = pd.read_csv('https://raw.githubusercontent.com/XianlaiYin/Data/main/CASA0007/data_clean/GEN.csv')
GDP = pd.read_csv('https://raw.githubusercontent.com/XianlaiYin/Data/main/CASA0007/data_clean/GDP.csv')
RES = pd.read_csv('https://raw.githubusercontent.com/XianlaiYin/Data/main/CASA0007/data_clean/RES.csv')
PRI = pd.read_csv('https://raw.githubusercontent.com/XianlaiYin/Data/main/CASA0007/data_clean/PRI.csv')
SAL = pd.read_csv('https://raw.githubusercontent.com/XianlaiYin/Data/main/CASA0007/data_clean/SAL.csv')
STO = pd.read_csv('https://raw.githubusercontent.com/XianlaiYin/Data/main/CASA0007/data_clean/STO.csv')
URBC = pd.read_csv('https://raw.githubusercontent.com/XianlaiYin/Data/main/CASA0007/data_clean/URBC.csv')

In [23]:
def min_max_scaler(x):
  return ((x-min(x))/(max(x)-min(x)))*100

def extract(i):
  POPi = POP[i]
  GDRi = GDR[i]
  URBi = URB[i]
  EDUi = EDU[i]
  FAMi = FAM[i]
  GENi = GEN[i]
  STOi = STO[i]
  GDPi = GDP[i]
  RESi = RES[i]
  PRIi = PRI[i]
  SALi = SAL[i]
  URBCi = URBC[i]

  POPii = min_max_scaler(POPi)  
  GDRii = min_max_scaler(GDRi)  
  URBii = min_max_scaler(URBi)  
  EDUii = min_max_scaler(EDUi)  
  FAMii = min_max_scaler(FAMi)  
  GENii = min_max_scaler(GENi)  
  STOii = min_max_scaler(STOi)  
  GDPii = min_max_scaler(GDPi)  
  RESii = min_max_scaler(RESi)  
  PRIii = min_max_scaler(PRIi) 
  SALii = min_max_scaler(SALi)  
  URBCii = min_max_scaler(URBCi)

  SETi1 = pd.DataFrame({'POP':POPi, 'URB':URBi, 'STO':STOi})
  SETi2 = pd.DataFrame({'GDR':GDRi, 'EDU':EDUi, 'FAM':FAMi, 'PRI':STOi, 'RES':RESi})
  
  return SETi1, SETi2

province = ['Shanghai','Jiangxi']

for i in province:
  SET = extract(i)
  SET1 = SET[0]
  SET2 = SET[1]

In [48]:
POP = pd.read_csv('https://raw.githubusercontent.com/XianlaiYin/Data/main/CASA0007/national/1990.csv', index_col=0)
HOU = pd.read_csv('https://raw.githubusercontent.com/XianlaiYin/Data/main/CASA0007/national/2000.csv', index_col=0)

HOU['UR'] = HOU['UP'] / HOU['TP']
HOU['ASARH'] = HOU['SARH'] / HOU['TP']
HOU['GEN'] = HOU['MP'] / HOU['FP']

In [49]:
#Def VIF
def drop_column_using_vif_(df, thresh=5):
    while True:
        df_with_const = add_constant(df)
        vif_df = pd.Series([variance_inflation_factor(df_with_const.values, i) 
               for i in range(df_with_const.shape[1])], name= "VIF",
              index=df_with_const.columns).to_frame()
        vif_df = vif_df.drop('const')
        if vif_df.VIF.max() > thresh:
            index_to_drop = vif_df.index[vif_df.VIF == vif_df.VIF.max()].tolist()[0]
            print('Dropping: {}'.format(index_to_drop))
            df = df.drop(columns = index_to_drop)
        else:
            break
    return df

#VIF
willing = HOU[['DPI', 'GDR']]
WILLING_VIF = drop_column_using_vif_(willing)

Dropping: DPI


c:\Users\xianl\AppData\Local\ESRI\conda\envs\arcpy\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [37]:
number = HOU[['LP','UP']]
NUMBER_VIF = drop_column_using_vif_(number)

Dropping: UP
Dropping: A1564


c:\Users\xianl\AppData\Local\ESRI\conda\envs\arcpy\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [38]:
OLS_y = HOU['TRBA'].apply(np.log)
OLS_x = HOU[['LP', 'UP']].apply(np.log)
OLSi = sm.OLS(endog = OLS_y, exog = sm.add_constant(OLS_x)).fit()
OLSi.summary()

c:\Users\xianl\AppData\Local\ESRI\conda\envs\arcpy\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   TRBA   R-squared:                       0.993
Model:                            OLS   Adj. R-squared:                  0.993
Method:                 Least Squares   F-statistic:                     1430.
Date:                Fri, 13 Jan 2023   Prob (F-statistic):           1.93e-21
Time:                        12:45:16   Log-Likelihood:                 37.273
No. Observations:                  22   AIC:                            -68.55
Df Residuals:                      19   BIC:                            -65.27
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         14.0796      9.548      1.475      0.157      -5.904      34.064
LP            -2.8495      0.944     -3.018      0.007      -4.826      -0.873
UP             2.8606      0.110     26.094      0.000       2.631       3.090
==============================================================================
Omnibus:                       11.300   Durbin-Watson:                   0.425
Prob(Omnibus):                  0.004   Jarque-Bera (JB):               10.142
Skew:                           1.132   Prob(JB):                      0.00628
Kurtosis:                       5.436   Cond. No.                     1.49e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.49e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [50]:
OLS_y = HOU['ASARH']
OLS_x = HOU[['GDR', 'DPI']]
OLSi = sm.OLS(endog = OLS_y, exog = sm.add_constant(OLS_x)).fit()
OLSi.summary()

c:\Users\xianl\AppData\Local\ESRI\conda\envs\arcpy\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  ASARH   R-squared:                       0.977
Model:                            OLS   Adj. R-squared:                  0.973
Method:                 Least Squares   F-statistic:                     253.2
Date:                Fri, 13 Jan 2023   Prob (F-statistic):           6.67e-15
Time:                        12:49:45   Log-Likelihood:                 34.885
No. Observations:                  22   AIC:                            -61.77
Df Residuals:                      18   BIC:                            -57.41
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -6.5658      5.724     -1.147      0.266     -18.592       5.461
GDR           -0.0276      0.005     -5.683      0.000      -0.038      -0.017
DPI         3.781e-05    3.7e-06     10.211      0.000       3e-05    4.56e-05
GEN            7.2997      5.494      1.329      0.201      -4.243      18.843
==============================================================================
Omnibus:                        1.256   Durbin-Watson:                   1.960
Prob(Omnibus):                  0.534   Jarque-Bera (JB):                1.116
Skew:                          -0.487   Prob(JB):                        0.572
Kurtosis:                       2.483   Cond. No.                     1.26e+07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.26e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""